# Ejercicios Pair Programming - Módulo 2 Sprint 2
## Lección 5 - ETL 3.2
### Natalia García Lago y Noemi Azorín

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [326]:
import pandas as pd
import mysql.connector
pd.options.display.max_columns = None

In [327]:
#lo que buscamos es que nos aparezca la primera columna como level_0, que va a ser por la que vamos a unir las tablas.

df_cargar = pd.read_csv('../ficheros/datos_union_clima_ataques.csv', index_col=0).reset_index() 
df_cargar.head(2)

,level_0,case_number,year,type,country,area,location,activity,name,age,time,species_,injury,fecha_limpia,fatal,sex,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,White shark,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
1,1,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,14h00 -15h00,NaN,Minor injury to left thigh,Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875


In [328]:
df_cargar.columns

Index(['level_0', 'case_number', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'age', 'time', 'species_', 'injury', 'fecha_limpia',
       'fatal', 'sex', 'pais', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb',
       'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb',
       'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb',
       'wind_950mb', 'wind_900mb', 'wind_850mb', 'wind_800mb', 'wind_750mb',
       'wind_700mb', 'wind_650mb', 'wind_600mb', 'wind_550mb', 'wind_500mb',
       'wind_450mb', 'wind_400mb', 'wind_350mb', 'wind_300mb', 'wind_250mb',
       'wind_200mb', 'index', 'timepoint', 'cloudcover', 'highcloud',
       'midcloud', 'lowcloud', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_amount', 'snow_depth', 'wind10m.direction',
       'wind10m.speed'],
      dtype='object')

In [329]:
#eliminamos las columnas que a primera vista no nos interesan aunque luego podemos seleccionar las columnas que queramos

df_cargar = df_cargar.drop(columns=['case_number','rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb',
       'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb',
       'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb',
       'wind_950mb', 'wind_900mb', 'wind_850mb', 'wind_800mb', 'wind_750mb',
       'wind_700mb', 'wind_650mb', 'wind_600mb', 'wind_550mb', 'wind_500mb',
       'wind_450mb', 'wind_400mb', 'wind_350mb', 'wind_300mb', 'wind_250mb',
       'wind_200mb', 'index'], axis = 1)

In [330]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        #  parámetros que fijos: el nombre de la BBDD y la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
    # método para crear tablas e insertar datos en ellas   
    def insertar_tabla_crear(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)       

### Queries de las tablas:

In [331]:
# conocemos el tipo de datos que tenemos para indicarlo en las queries para cada dato

df_cargar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4395 entries, 0 to 4394
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   level_0            4395 non-null   int64  
 1   year               4395 non-null   float64
 2   type               4394 non-null   object 
 3   country            4395 non-null   object 
 4   area               4363 non-null   object 
 5   location           4276 non-null   object 
 6   activity           4066 non-null   object 
 7   name               4325 non-null   object 
 8   age                2681 non-null   object 
 9   time               2395 non-null   object 
 10  species_           2647 non-null   object 
 11  injury             4381 non-null   object 
 12  fecha_limpia       4148 non-null   object 
 13  fatal              3975 non-null   object 
 14  sex                4059 non-null   object 
 15  pais               4395 non-null   object 
 16  timepoint          4395 

### Creamos las tablas con las columnas que queremos que tenga nuestra BD:

In [332]:
tabla_ataques = """ CREATE TABLE IF NOT EXISTS `tabla_ataques`(
                    `level_0` FLOAT NOT NULL,
                    `type` VARCHAR(50),
                    `country` VARCHAR(50),
                    `age` VARCHAR(50),
                    `species_` VARCHAR(50),
                    `fatal` VARCHAR(50),
                    `sex` VARCHAR(50),
                    `year` FLOAT,
                    PRIMARY KEY (`level_0`))
                    ENGINE = InnoDB; """

In [333]:
tabla_clima = """ CREATE TABLE IF NOT EXISTS `tabla_clima`(
                    `level_0` FLOAT NOT NULL,
                    `timepoint` INT,
                    `cloudcover` INT,
                    `highcloud` INT,
                    `midcloud` INT,
                    `lowcloud` INT,
                    `temp2m` INT,
                    `lifted_index` INT,
                    `rh2m` VARCHAR(50),
                    `msl_pressure` INT,
                    `prec_amount` INT,
                    `snow_depth` INT,
                    `wind10m.direction` VARCHAR(50),
                    `wind10m.speed` INT,
                    `pais` VARCHAR(50),
                    PRIMARY KEY (`level_0`),
                    CONSTRAINT `fk_ataques_clima`
                    FOREIGN KEY (`level_0`)
                    REFERENCES `tabla_ataques` (`level_0`))
                    ENGINE = InnoDB; """

### Llamamos a la clase:

In [334]:
carga = Cargar("tiburones_clase2", "AlumnaAdalab")

In [335]:
carga.crear_bbdd()

Conexión realizada con éxito


### Creamos las tablas:

In [336]:
carga.insertar_tabla_crear(tabla_ataques)
carga.insertar_tabla_crear(tabla_clima)

### Insertamos los datos de dos columnas de la tabla ataques:

In [337]:
for indice, fila in df_cargar.iterrows():
    query_ataques = f""" INSERT INTO `tabla_ataques`(`level_0`, `type` )
                         VALUES ('{fila["level_0"]}', '{fila["type"]}');
                    """
    carga.insertar_tabla_crear(query_ataques)

### Insertamos los datos de dos columnas de la tabla clima:

In [338]:
for indice, fila in df_cargar.iterrows(): 
    
    query_clima = f""" INSERT INTO `tabla_clima` (`level_0`,`timepoint`)
            VALUES ( "{fila['level_0']}", "{fila['timepoint']}");
            """
    carga.insertar_tabla_crear(query_clima)